# Pengerjaan Tugas Deep Learning menggunakan PyTorch and TensorFlow (Week 7, WMT Dataset Menggunakan Encoder-to-Decoder LSTM)

## Persiapan: Instalasi library

### 1. Memastikan Instalasi library

In [ ]:
# !pip install numpy matplotlib scikit-learn torch tensorflow
# %pip install numpy matplotlib scikit-learn torch tensorflow[and-cuda] keras-tuner nltk imbalanced-learn

%pip install keras-tuner imbalanced-learn datasets optuna transformers

### 2. Mengimpor Library yang Dibutuhkan

In [ ]:
# Import library yang diperlukan
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torch.nn.utils.rnn import pad_sequence
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.layers import LSTM, Bidirectional, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from datasets import load_dataset
import keras_tuner as kt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import optuna
from transformers import AutoTokenizer
import time

# Memeriksa apakah GPU tersedia dan dapat digunakan oleh PyTorch
gpu_available = torch.cuda.is_available()
print(f"GPU available: {gpu_available}")

# Memeriksa apakah GPU tersedia dan dapat digunakan oleh TensorFlow
gpu_available = tf.test.is_gpu_available()
print(f"GPU available: {gpu_available}")


### 3. Mendefinisikan Parameter dan Pre-processing Dataset

In [ ]:
# Konstanta dan parameter
NUM_WORDS = 40000
BATCH_SIZE = 16
EPOCHS = 25

# Set seed untuk reproduktivitas
SEED = 42
torch.manual_seed(SEED)
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# Memuat dataset
print("Loading dataset...")
dataset = load_dataset("wmt14", "de-en")
print(f"Dataset loaded with {len(dataset['train'])} training examples.")

# Melihat contoh data
print("Sample data:")
for i in range(3):
    print(f"German: {dataset['train'][i]['translation']['de']}")
    print(f"English: {dataset['train'][i]['translation']['en']}")
    print()

train_dataset = dataset['train'].select(range(min(NUM_WORDS, len(dataset['train']))))
val_dataset = dataset['validation'].select(range(min(5000, len(dataset['validation']))))
test_dataset = dataset['test'].select(range(min(5000, len(dataset['test']))))

print(f"Using {len(train_dataset)} samples for training.")
print(f"Using {len(val_dataset)} samples for validation.")
print(f"Using {len(test_dataset)} samples for testing.")

# Inisialisasi tokenizer
tokenizer_de = AutoTokenizer.from_pretrained("google/mt5-small")
tokenizer_en = AutoTokenizer.from_pretrained("google/mt5-small")

# Fungsi untuk tokenisasi dan padding
def tokenize_and_prepare(examples, src_lang='de', tgt_lang='en', max_length=128):
    source_texts = [example['translation'][src_lang] for example in examples]
    target_texts = [example['translation'][tgt_lang] for example in examples]
    
    # Tokenisasi
    source_encodings = tokenizer_de(source_texts, truncation=True, max_length=max_length, padding='max_length', return_tensors='pt')
    target_encodings = tokenizer_en(target_texts, truncation=True, max_length=max_length, padding='max_length', return_tensors='pt')
    
    return {
        'input_ids': source_encodings.input_ids,
        'attention_mask': source_encodings.attention_mask,
        'labels': target_encodings.input_ids,
        'decoder_attention_mask': target_encodings.attention_mask
    }


### 4. Menyiapkan Fungsi Evaluasi

In [ ]:
# Fungsi untuk menghitung metrik evaluasi
def calculate_metrics(predictions, targets, binary=False):
    metrics = {}
    
    if binary:
        # Binary classification metrics
        metrics['accuracy'] = accuracy_score(targets, predictions)
        metrics['precision'] = precision_score(targets, predictions)
        metrics['recall'] = recall_score(targets, predictions)
        metrics['f1'] = f1_score(targets, predictions)
        
        # ROC AUC
        try:
            metrics['auc'] = roc_auc_score(targets, predictions)
            fpr, tpr, _ = roc_curve(targets, predictions)
            metrics['roc'] = (fpr, tpr)
        except:
            metrics['auc'] = 0
            metrics['roc'] = ([], [])
    else:
        # Multi-class metrics (untuk NMT, ini akan menjadi per-token)
        # Ubah menjadi binary task dengan membandingkan prediksi dan target
        binary_accuracy = (np.array(predictions) == np.array(targets)).astype(int)
        metrics['accuracy'] = np.mean(binary_accuracy)
        
        # Untuk NMT, presisi, recall dan F1 score kurang relevan per token
        # tapi bisa dihitung untuk memberikan informasi
        # predictions dan targets harus diubah menjadi one-hot encoding untuk metrik ini
        # Untuk kesederhanaan, gunakan accuracy sebagai proxy
        metrics['precision'] = metrics['accuracy']
        metrics['recall'] = metrics['accuracy']
        metrics['f1'] = metrics['accuracy']
        metrics['auc'] = metrics['accuracy']  # Placeholder
        metrics['roc'] = ([], [])  # Placeholder
    
    return metrics

# Fungsi untuk evaluasi model PyTorch
def evaluate_model_pytorch(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for batch in dataloader:
            src = batch['input_ids'].to(device)
            trg = batch['labels'].to(device)
            
            output = model(src, trg, 0)  # Turn off teacher forcing
            
            # Abaikan token awal
            output = output[:, 1:].reshape(-1, output.shape[-1])
            trg = trg[:, 1:].reshape(-1)
            
            loss = criterion(output, trg)
            epoch_loss += loss.item()
            
            # Kumpulkan prediksi dan target untuk metrik evaluasi
            predictions = output.argmax(1).cpu().numpy()
            targets = trg.cpu().numpy()
            
            # Hanya menggunakan token valid (bukan padding)
            valid_indices = targets != 0  # Asumsi 0 adalah padding
            all_predictions.extend(predictions[valid_indices])
            all_targets.extend(targets[valid_indices])
    
    # Hitung metrik evaluasi
    metrics = calculate_metrics(all_predictions, all_targets)
    
    return epoch_loss / len(dataloader), metrics

In [ ]:
# Fungsi untuk evaluasi model TensorFlow
def evaluate_model_tf(model, test_data, model_name):
    x_test, y_test = test_data
    
    # Prediksi
    y_pred_prob = model.predict(x_test)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()
    
    # Menghitung metrik evaluasi
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_prob)
    
    # Menghitung ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    
    # Print hasil evaluasi
    print(f"\nEvaluasi Model {model_name}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC: {auc:.4f}")
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'fpr': fpr,
        'tpr': tpr,
        'y_pred_prob': y_pred_prob,
        'y_pred': y_pred,
        'y_test': y_test
    }

### 5. Menyiapkan Fungsi Visualisasi

In [ ]:
# Fungsi untuk membuat visualisasi hasil
# Fungsi untuk membuat visualisasi training history
def plot_training_history(history, model_name):
    plt.figure(figsize=(12, 5))
    
    best_epoch = history['val_loss'].index(min(history['val_loss']))
    best_val_loss = min(history['val_loss'])

    # Plot Loss, dengan tanda pada titik terbaik
    plt.subplot(1, 2, 1)
    plt.plot(history['loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Valid Loss')
    plt.axvline(x=best_epoch, color='r', linestyle='--', label='Best Model')
    plt.axhline(y=best_val_loss, color='g', linestyle='--', label='Best Val Loss')
    plt.title(f'{model_name} - Loss History')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history['accuracy'], label='Train Accuracy')
    plt.plot(history['val_accuracy'], label='Valid Accuracy')
    plt.axvline(x=best_epoch, color='r', linestyle='--', label='Best Model')
    plt.axhline(y=history['val_accuracy'][best_epoch], color='g', linestyle='--', label='Val Accuracy at Best Model')
    plt.title(f'{model_name} - Accuracy History')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(f'training_history_tensorflow_{model_name}.png')
    plt.show()

# Fungsi untuk membuat confusion matrix
def plot_confusion_matrix(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(f'{model_name} - Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, ['Negative', 'Positive'])
    plt.yticks(tick_marks, ['Negative', 'Positive'])
    
    thresh = cm.max() / 2.
    for i, j in np.ndindex(cm.shape):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig(f'confusion_matrix_tensorflow_{model_name}.png')
    plt.show()

# Fungsi untuk membuat ROC curve
def plot_roc_curve(results_dict):
    plt.figure(figsize=(10, 8))
    
    for model_name, result in results_dict.items():
        plt.plot(result['fpr'], result['tpr'], label=f'{model_name} (AUC = {result["auc"]:.4f})')
    
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve Comparison')
    plt.legend()
    plt.grid(True)
    plt.savefig('roc_curve_comparison_tensorflow.png')
    plt.show()

# Fungsi untuk membuat bar chart perbandingan metrik
def plot_metrics_comparison(results_dict):
    metrics = ['accuracy', 'precision', 'recall', 'f1', 'auc']
    models = list(results_dict.keys())
    
    values = {metric: [results_dict[model][metric] for model in models] for metric in metrics}
    
    plt.figure(figsize=(12, 8))
    bar_width = 0.15
    index = np.arange(len(models))
    
    for i, metric in enumerate(metrics):
        plt.bar(index + i * bar_width, values[metric], bar_width, 
                label=metric.capitalize())
    
    plt.xlabel('Models')
    plt.ylabel('Scores')
    plt.title('Performance Metrics Comparison')
    plt.xticks(index + bar_width * 2, models)
    plt.legend()
    plt.grid(True, axis='y')
    plt.savefig('metrics_comparison_tensorflow.png')
    plt.show()


## Model Encoder-to-Decoder LSTM Menggunakan PyTorch

### 1. Penyusunan Dataset dan Loader untuk PyTorch

In [ ]:
# Class untuk dataset PyTorch
class TranslationDataset(Dataset):
    def __init__(self, examples, src_lang='de', tgt_lang='en', max_length=128):
        self.examples = examples
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang
        self.max_length = max_length
        
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, idx):
        example = self.examples[idx]
        source_text = example['translation'][self.src_lang]
        target_text = example['translation'][self.tgt_lang]
        
        # Tokenisasi
        source_tokens = tokenizer_de(source_text, truncation=True, max_length=self.max_length, return_tensors='pt')
        target_tokens = tokenizer_en(target_text, truncation=True, max_length=self.max_length, return_tensors='pt')
        
        return {
            'input_ids': source_tokens.input_ids.squeeze(),
            'attention_mask': source_tokens.attention_mask.squeeze(),
            'labels': target_tokens.input_ids.squeeze(),
            'decoder_attention_mask': target_tokens.attention_mask.squeeze()
        }

# Membuat dataset untuk PyTorch
train_dataset_torch = TranslationDataset(train_dataset)
val_dataset_torch = TranslationDataset(val_dataset)
test_dataset_torch = TranslationDataset(test_dataset)

# Collate function untuk DataLoader PyTorch
def collate_fn(batch):
    input_ids = pad_sequence([item['input_ids'] for item in batch], batch_first=True)
    attention_mask = pad_sequence([item['attention_mask'] for item in batch], batch_first=True)
    labels = pad_sequence([item['labels'] for item in batch], batch_first=True)
    decoder_attention_mask = pad_sequence([item['decoder_attention_mask'] for item in batch], batch_first=True)
    
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels,
        'decoder_attention_mask': decoder_attention_mask
    }

# DataLoader untuk PyTorch
batch_size = 32
train_loader = DataLoader(train_dataset_torch, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset_torch, batch_size=batch_size, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset_torch, batch_size=batch_size, collate_fn=collate_fn)

### 2. Menyusun Model Encoder-Decoder LSTM


In [ ]:
# Model Encoder-Decoder PyTorch
class EncoderDecoder(nn.Module):
    def __init__(self, input_dim, output_dim, enc_emb_dim, dec_emb_dim, enc_hidden_dim, 
                 dec_hidden_dim, enc_dropout, dec_dropout, device):
        super().__init__()
        
        self.encoder = Encoder(input_dim, enc_emb_dim, enc_hidden_dim, dec_hidden_dim, enc_dropout, device)
        self.decoder = Decoder(output_dim, dec_emb_dim, enc_hidden_dim, dec_hidden_dim, dec_dropout, device)
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [batch_size, src_len]
        # trg: [batch_size, trg_len]
        
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        
        # Tensor untuk menyimpan output
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
        
        # Enkode sumber
        encoder_outputs, hidden = self.encoder(src)
        
        # Dekode target
        # Gunakan token awal sebagai input awal decoder
        input = trg[:, 0]
        
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            outputs[:, t] = output
            
            # Putuskan apakah menggunakan prediksi atau target sebenarnya sebagai input berikutnya
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[:, t] if teacher_force else top1
            
        return outputs

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hidden_dim, dec_hidden_dim, dropout, device):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(enc_hidden_dim * 2, dec_hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.device = device
        
    def forward(self, src):
        # src: [batch_size, src_len]
        
        embedded = self.dropout(self.embedding(src))
        # embedded: [batch_size, src_len, emb_dim]
        
        outputs, hidden = self.rnn(embedded)
        # outputs: [batch_size, src_len, enc_hidden_dim * 2]
        # hidden: [2, batch_size, enc_hidden_dim]
        
        # Gabungkan hidden state forward dan backward terakhir
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        # hidden: [batch_size, enc_hidden_dim * 2]
        
        hidden = torch.tanh(self.fc(hidden))
        # hidden: [batch_size, dec_hidden_dim]
        
        return outputs, hidden.unsqueeze(0)

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hidden_dim, dec_hidden_dim, dropout, device):
        super().__init__()
        
        self.output_dim = output_dim
        self.attention = Attention(enc_hidden_dim * 2, dec_hidden_dim)
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim + enc_hidden_dim * 2, dec_hidden_dim, batch_first=True)
        self.fc_out = nn.Linear(dec_hidden_dim + enc_hidden_dim * 2 + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.device = device
        
    def forward(self, input, hidden, encoder_outputs):
        # input: [batch_size]
        # hidden: [1, batch_size, dec_hidden_dim]
        # encoder_outputs: [batch_size, src_len, enc_hidden_dim * 2]
        
        input = input.unsqueeze(1)  # [batch_size, 1]
        
        embedded = self.dropout(self.embedding(input))
        # embedded: [batch_size, 1, emb_dim]
        
        # Perhitungan attention
        a = self.attention(hidden, encoder_outputs)
        # a: [batch_size, 1, src_len]
        
        # Weighted sum encoder outputs
        weighted = torch.bmm(a, encoder_outputs)
        # weighted: [batch_size, 1, enc_hidden_dim * 2]
        
        # Konkatenasi embedding dan weighted untuk input RNN
        rnn_input = torch.cat((embedded, weighted), dim=2)
        # rnn_input: [batch_size, 1, emb_dim + enc_hidden_dim * 2]
        
        output, hidden = self.rnn(rnn_input, hidden)
        # output: [batch_size, 1, dec_hidden_dim]
        # hidden: [1, batch_size, dec_hidden_dim]
        
        # Konkatenasi untuk prediksi
        output = torch.cat((output.squeeze(1), weighted.squeeze(1), embedded.squeeze(1)), dim=1)
        # output: [batch_size, dec_hidden_dim + enc_hidden_dim * 2 + emb_dim]
        
        prediction = self.fc_out(output)
        # prediction: [batch_size, output_dim]
        
        return prediction, hidden

class Attention(nn.Module):
    def __init__(self, enc_hidden_dim, dec_hidden_dim):
        super().__init__()
        
        self.attn = nn.Linear(enc_hidden_dim + dec_hidden_dim, dec_hidden_dim)
        self.v = nn.Linear(dec_hidden_dim, 1, bias=False)
        
    def forward(self, hidden, encoder_outputs):
        # hidden: [1, batch_size, dec_hidden_dim]
        # encoder_outputs: [batch_size, src_len, enc_hidden_dim]
        
        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]
        
        # Ulangi hidden state untuk setiap token sumber
        hidden = hidden.permute(1, 0, 2)  # [batch_size, 1, dec_hidden_dim]
        hidden = hidden.repeat(1, src_len, 1)  # [batch_size, src_len, dec_hidden_dim]
        
        # Konkatenasi hidden state dengan encoder outputs
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        # energy: [batch_size, src_len, dec_hidden_dim]
        
        attention = self.v(energy).squeeze(2)
        # attention: [batch_size, src_len]
        
        return torch.softmax(attention, dim=1).unsqueeze(1)

### 3. Menyusun Fungsi Hyperparameter Tuning

In [ ]:
# Fungsi untuk hyperparameter tuning dengan Optuna untuk PyTorch
def objective_pytorch(trial):
    # Hyperparameters untuk tuning
    enc_emb_dim = trial.suggest_int('enc_emb_dim', 128, 512)
    dec_emb_dim = trial.suggest_int('dec_emb_dim', 128, 512)
    enc_hidden_dim = trial.suggest_int('enc_hidden_dim', 128, 512)
    dec_hidden_dim = trial.suggest_int('dec_hidden_dim', 128, 512)
    enc_dropout = trial.suggest_float('enc_dropout', 0.1, 0.5)
    dec_dropout = trial.suggest_float('dec_dropout', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    
    # Buat model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    input_dim = len(tokenizer_de.get_vocab())
    output_dim = len(tokenizer_en.get_vocab())
    
    model = EncoderDecoder(input_dim, output_dim, enc_emb_dim, dec_emb_dim, 
                         enc_hidden_dim, dec_hidden_dim, enc_dropout, dec_dropout, device).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # Abaikan padding
    
    # Training dengan early stopping
    best_val_loss = float('inf')
    patience = 3
    patience_counter = 0
    n_epochs = 10
    
    for epoch in range(n_epochs):
        model.train()
        epoch_loss = 0
        
        for batch in train_loader:
            optimizer.zero_grad()
            
            src = batch['input_ids'].to(device)
            trg = batch['labels'].to(device)
            
            output = model(src, trg)
            
            # Hilangkan token awal pada output dan target
            output = output[:, 1:].reshape(-1, output.shape[-1])
            trg = trg[:, 1:].reshape(-1)
            
            loss = criterion(output, trg)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            epoch_loss += loss.item()
        
        train_loss = epoch_loss / len(train_loader)
        
        # Evaluate on validation set
        val_loss, val_metrics = evaluate_model_pytorch(model, val_loader, criterion, device)
        
        print(f'Epoch: {epoch+1}')
        print(f'Train Loss: {train_loss:.3f}')
        print(f'Val Loss: {val_loss:.3f}')
        print(f'Val Accuracy: {val_metrics["accuracy"]:.3f}')
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model_pytorch.pt')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered!")
                break
    
    # Load best model for final evaluation
    model.load_state_dict(torch.load('best_model_pytorch.pt'))
    _, test_metrics = evaluate_model_pytorch(model, test_loader, criterion, device)
    
    return test_metrics['accuracy']

print("Running PyTorch model hyperparameter tuning...")
study_pytorch = optuna.create_study(direction='maximize')
study_pytorch.optimize(objective_pytorch, n_trials=5)  # Adjust n_trials as needed

best_params_pytorch = study_pytorch.best_params
print("\nBest PyTorch hyperparameters:")
print(best_params_pytorch)
print(f"Best accuracy: {study_pytorch.best_value:.4f}")


### 4. Melatih Model dengan Hyperparameter Terbaik

In [ ]:
# Train final PyTorch model
print("\nTraining final PyTorch model with best hyperparameters...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_dim = len(tokenizer_de.get_vocab())
output_dim = len(tokenizer_en.get_vocab())

model_pytorch = EncoderDecoder(
    input_dim, output_dim, 
    best_params_pytorch['enc_emb_dim'], 
    best_params_pytorch['dec_emb_dim'],
    best_params_pytorch['enc_hidden_dim'], 
    best_params_pytorch['dec_hidden_dim'],
    best_params_pytorch['enc_dropout'], 
    best_params_pytorch['dec_dropout'], 
    device
).to(device)

optimizer = optim.Adam(model_pytorch.parameters(), lr=best_params_pytorch['learning_rate'])
criterion = nn.CrossEntropyLoss(ignore_index=0)

history_pytorch = {'accuracy': [], 'loss': [], 'val_accuracy': [], 'val_loss': []}
n_epochs = 20
best_val_loss = float('inf')

for epoch in range(n_epochs):
    # Training
    model_pytorch.train()
    epoch_loss = 0
    
    for batch in train_loader:
        optimizer.zero_grad()
        
        src = batch['input_ids'].to(device)
        trg = batch['labels'].to(device)
        
        output = model_pytorch(src, trg)
        
        output = output[:, 1:].reshape(-1, output.shape[-1])
        trg = trg[:, 1:].reshape(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_pytorch.parameters(), 1.0)
        optimizer.step()
        
        epoch_loss += loss.item()
    
    train_loss = epoch_loss / len(train_loader)

    # Validation
    val_loss, val_metrics = evaluate_model_pytorch(model_pytorch, val_loader, criterion, device)

    print(f'Epoch: {epoch+1}')
    print(f'Train Loss: {train_loss:.3f}')
    print(f'Val Loss: {val_loss:.3f}')
    print(f'Val Accuracy: {val_metrics["accuracy"]:.3f}')

    history_pytorch['accuracy'].append(val_metrics['accuracy'])
    history_pytorch['loss'].append(train_loss)
    history_pytorch['val_accuracy'].append(val_metrics['accuracy'])
    history_pytorch['val_loss'].append(val_loss)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model_pytorch.state_dict(), 'final_model_pytorch.pt')

### 5. Mengevaluasi Model

In [ ]:
# Load model terbaik dan evaluasi
model_pytorch.load_state_dict(torch.load('final_model_pytorch.pt'))
test_loss, test_metrics_pytorch = evaluate_model_pytorch(pytorch, test_loader, criterion, device)

print("\nFinal PyTorch model evaluation:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_metrics_pytorch['accuracy']:.4f}")
print(f"Test Precision: {test_metrics_pytorch['precision']:.4f}")
print(f"Test Recall: {test_metrics_pytorch['recall']:.4f}")
print(f"Test F1 Score: {test_metrics_pytorch['f1']:.4f}")
print(f"Test AUC: {test_metrics_pytorch['auc']:.4f}")

### 6. Memvisualisasikan Prediksi dari model yang telah dilatih

In [ ]:
# Plotting hasil pelatihan
plot_training_history(history_pytorch.history, "birnn")

# Plotting confusion matrix
plot_confusion_matrix(test_metrics_pytorch['y_test'], test_metrics_pytorch['y_pred'], "pytorch")

### 7. Menyimpan Model

In [ ]:
# Menyimpan model
#torch.save(final_model_pytorch.state_dict(), 'final_model_pytorch.pt')
#print("Model disimpan sebagai 'final_model_pytorch.pt'.")

## Model Encoder-to-Decoder LSTM Menggunakan TensorFlow

### 1. Konversi Dataset ke format TensorFlow

In [ ]:
# Fungsi untuk mengkonversi dataset ke format TensorFlow
def prepare_tf_dataset(dataset, batch_size=32, buffer_size=10000):
    def gen():
        for item in dataset:
            yield {
                'encoder_inputs': tokenizer_de.encode(item['translation']['de'], max_length=128, truncation=True),
                'decoder_inputs': tokenizer_en.encode(item['translation']['en'], max_length=128, truncation=True)
            }
    
    def map_func(item):
        return {
            'encoder_inputs': item['encoder_inputs'],
            'decoder_inputs': item['decoder_inputs'][:-1]  # Input tanpa token akhir
        }, item['decoder_inputs'][1:]  # Target dimulai dari token kedua
    
    ds = tf.data.Dataset.from_generator(
        gen,
        output_signature={
            'encoder_inputs': tf.TensorSpec(shape=(None,), dtype=tf.int32),
            'decoder_inputs': tf.TensorSpec(shape=(None,), dtype=tf.int32)
        }
    )
    
    ds = ds.map(map_func)
    ds = ds.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return ds

### 2. Menyusun Model Encode-to-Decoder LSTM


In [ ]:
# Model TensorFlow/Keras untuk NMT
def create_tensorflow_model(vocab_size_src, vocab_size_tgt, embedding_dim, hidden_units, dropout_rate):
    # Encoder
    encoder_inputs = Input(shape=(None,))
    encoder_embedding = Embedding(vocab_size_src, embedding_dim)(encoder_inputs)
    encoder_dropout = Dropout(dropout_rate)(encoder_embedding)
    encoder_lstm = Bidirectional(LSTM(hidden_units, return_sequences=True, return_state=True))
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_dropout)
    
    # Mengkombinasikan state dari kedua arah
    state_h = Concatenate()([forward_h, backward_h])
    state_c = Concatenate()([forward_c, backward_c])
    encoder_states = [state_h, state_c]
    
    # Decoder
    decoder_inputs = Input(shape=(None,))
    decoder_embedding = Embedding(vocab_size_tgt, embedding_dim)(decoder_inputs)
    decoder_dropout = Dropout(dropout_rate)(decoder_embedding)
    
    # Menggunakan LSTM yang sama dengan dimensi hidden state yang digandakan karena bidirectional encoder
    decoder_lstm = LSTM(hidden_units * 2, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_dropout, initial_state=encoder_states)
    
    # Attention mechanism
    attention = Attention()([decoder_outputs, encoder_outputs])
    concat = Concatenate()([decoder_outputs, attention])
    
    # Output layer
    decoder_dense = Dense(vocab_size_tgt, activation='softmax')
    decoder_outputs = decoder_dense(concat)
    
    # Define model
    model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    return model

### 2. Menyusun Fungsi Hyperparameter Tuning

In [ ]:
# Fungsi untuk hyperparameter tuning dengan Optuna untuk TensorFlow
def objective_tensorflow(trial):
    # Hyperparameters untuk tuning
    embedding_dim = trial.suggest_int('embedding_dim', 128, 512)
    hidden_units = trial.suggest_int('hidden_units', 128, 512)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    
    # Persiapkan dataset
    train_ds_tf = prepare_tf_dataset(train_dataset)
    val_ds_tf = prepare_tf_dataset(val_dataset)
    
    # Buat model
    vocab_size_src = len(tokenizer_de.get_vocab())
    vocab_size_tgt = len(tokenizer_en.get_vocab())
    
    model = create_tensorflow_model(vocab_size_src, vocab_size_tgt, embedding_dim, hidden_units, dropout_rate)
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint('best_model_tf.h5', monitor='val_loss', save_best_only=True)
    
    # Train model
    history = model.fit(
        train_ds_tf,
        epochs=10,
        validation_data=val_ds_tf,
        callbacks=[early_stopping, model_checkpoint]
    )
    
    # Evaluate model
    test_ds_tf = prepare_tf_dataset(test_dataset)
    _, test_accuracy = model.evaluate(test_ds_tf)
    
    return test_accuracy

print("\nRunning TensorFlow model hyperparameter tuning...")
study_tf = optuna.create_study(direction='maximize')
study_tf.optimize(objective_tensorflow, n_trials=5)  # Adjust n_trials as needed

best_params_tf = study_tf.best_params
print("\nBest TensorFlow hyperparameters:")
print(best_params_tf)
print(f"Best accuracy: {study_tf.best_value:.4f}")

### 4. Melatih Model dengan Hyperparameter Terbaik

In [ ]:
# Train final TensorFlow model
print("\nTraining final TensorFlow model with best hyperparameters...")
vocab_size_src = len(tokenizer_de.get_vocab())
vocab_size_tgt = len(tokenizer_en.get_vocab())

model_tf = create_tensorflow_model(
    vocab_size_src, vocab_size_tgt,
    best_params_tf['embedding_dim'],
    best_params_tf['hidden_units'],
    best_params_tf['dropout_rate']
)

model_tf.compile(
    optimizer=tf.keras.optimizers.Adam(best_params_tf['learning_rate']),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

train_ds_tf = prepare_tf_dataset(train_dataset)
val_ds_tf = prepare_tf_dataset(val_dataset)
test_ds_tf = prepare_tf_dataset(test_dataset)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('final_model_tf.h5', monitor='val_loss', save_best_only=True)

history_tf = model_tf.fit(
    train_ds_tf,
    epochs=20,
    validation_data=val_ds_tf,
    callbacks=[early_stopping, model_checkpoint]
)

### 3. Mengevaluasi Model dengan Menghitung Akurasi, Presisi, Recall, F1Squared, ROC, dan AUC-ROC

In [ ]:
# Evaluasi model RNN
result_tf = evaluate_model_tf(model_tf, test_ds_tf, "TensorFlow")

### 4. Memvisualisasikan Prediksi dari model yang telah dilatih

In [ ]:
# Plotting hasil pelatihan
plot_training_history(history_tf.history, "TensorFlow")

# Plotting confusion matrix
plot_confusion_matrix(result_tf['y_test'], result_tf['y_pred'], "TensorFlow")

### 5. Menyimpan Model

In [ ]:
# Menyimpan model
model_tf.save('model_tf.keras')
print("Model disimpan sebagai 'model_tf.keras'.")

## Perbandingan antara model

In [ ]:
# Menghitung perbandingan untuk semua model
results_dict = {
    'PyTorch': test_metrics_pytorch,
    'TensorFlow': result_tf
}

# Plotting ROC curve
plot_roc_curve(results_dict)
# Plotting perbandingan metrik
plot_metrics_comparison(results_dict)